In [ ]:
# For Jai: this contains the new model with reward information

import nest_asyncio
nest_asyncio.apply()

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import hddCRP.dataLoader as dl
import hddCRP.newContextModel2 as crp
import os

In [ ]:
print(f"groups for population model: {dl.grp_names_all}")

results_directory = "Results/RewardModel/"
if(not os.path.exists(results_directory)):
    os.makedirs(results_directory)

OVERWRITE = False

N_trials = 50

In [ ]:
print("fitting individual animals")
fit_file = f"{results_directory}/individual_fit"
fit_summary_file = f"{results_directory}/individual_fit_summary"
fit_file += f".pkl"
fit_summary_file  += f".pkl"

if(((not os.path.isfile(fit_file)) or (not os.path.isfile(fit_summary_file))) or OVERWRITE):
    data_fits = pd.DataFrame()
    data_fit_metrics = pd.DataFrame()
else:
    data_fits = pd.read_pickle(fit_file)
    data_fit_metrics = pd.read_pickle(fit_summary_file)


for ii, subject in enumerate(dl.get_subjects()):
    if( ("subject" in data_fit_metrics) and data_fit_metrics.query("subject == @subject").size > 0):
        print(f"found subject {ii}: {subject}")
        continue

    print(f"fitting subject {ii}: {subject}")
    rs = 1400 + ii;
    model = crp.create_indiv_model(subject) # NOTE: follow this function to see the data loading and model construction calls
    model.build(random_seed = rs)
    model.fit_model()
    fit_df = model.fit.to_frame()
    fit_df["subject"] = subject
    fit_df["group"] = subject
    summary_df = model.fit_summary()
    summary_df["subject"] = subject
    summary_df["group"] = subject

    data_fit_metrics = pd.concat([data_fit_metrics,summary_df], copy=False)
    data_fits = pd.concat([data_fits,fit_df], copy=False)

    data_fits.to_pickle(fit_file)
    data_fit_metrics.to_pickle(fit_summary_file)

In [ ]:
print("fitting groups of animals")
fit_file = f"{results_directory}/fit"
fit_summary_file = f"{results_directory}/fit_summary"
fit_file += f".pkl"
fit_summary_file  += f".pkl"

if(((not os.path.isfile(fit_file)) or (not os.path.isfile(fit_summary_file))) or OVERWRITE):
    data_fits = pd.DataFrame()
    data_fit_metrics = pd.DataFrame()
else:
    data_fits = pd.read_pickle(fit_file)
    data_fit_metrics = pd.read_pickle(fit_summary_file)


for ii, group in enumerate(dl.grp_names_all):
    if( ("group" in data_fit_metrics) and data_fit_metrics.query("group == @group").size > 0):
        print(f"found group {ii}: {group}")
        continue
    
    print(f"fitting group {ii}: {group}")
    rs = 1200 + ii;
    model = crp.create_pop_model(group, n_trials=N_trials) # NOTE: follow this function to see the data loading and model construction calls
    model.build(random_seed = rs)
    model.fit_model()
    fit_df = model.fit.to_frame()
    fit_df["group"] = group
    fit_df["subject"] = group
    summary_df = model.fit_summary()
    summary_df["group"] = group
    summary_df["subject"] = group

    data_fit_metrics = pd.concat([data_fit_metrics,summary_df], copy=False)
    data_fits = pd.concat([data_fits,fit_df], copy=False)

    data_fits.to_pickle(fit_file)
    data_fit_metrics.to_pickle(fit_summary_file)